In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [5]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [6]:
df_clean=pd.get_dummies(df[['Survived','Pclass','Sex','Age','Fare','Embarked']],drop_first=True)
df_clean

,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,1,0,1
1,1,1,38.0,71.2833,0,0,0
2,1,3,26.0,7.9250,0,0,1
3,1,1,35.0,53.1000,0,0,1
4,0,3,35.0,8.0500,1,0,1
...,...,...,...,...,...,...,...
886,0,2,27.0,13.0000,1,0,1
887,1,1,19.0,30.0000,0,0,1
888,0,3,NaN,23.4500,0,0,1
889,1,1,26.0,30.0000,1,0,0


In [7]:
df_clean.isnull().sum()

Survived        0
Pclass          0
Age           177
Fare            0
Sex_male        0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [8]:
df['Survived'].value_counts()/df.shape[0]

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [9]:
df_clean['Age'].fillna(df['Age'].median(),inplace=True)

In [10]:
df_clean.isnull().sum()

Survived      0
Pclass        0
Age           0
Fare          0
Sex_male      0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [11]:
df_clean.head()

,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,1,0,1
1,1,1,38.0,71.2833,0,0,0
2,1,3,26.0,7.9250,0,0,1
3,1,1,35.0,53.1000,0,0,1
4,0,3,35.0,8.0500,1,0,1


In [12]:
df_clean.columns

Index(['Survived', 'Pclass', 'Age', 'Fare', 'Sex_male', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

In [13]:
df_clean.isnull().sum()

Survived      0
Pclass        0
Age           0
Fare          0
Sex_male      0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test =train_test_split(df_clean[['Pclass','Sex_male','Age','Fare','Embarked_Q','Embarked_S']],\
                                                   df_clean['Survived'],test_size=0.20,random_state=1)

In [21]:
X_test.shape

(179, 6)

In [22]:
y_train.value_counts()

0    443
1    269
Name: Survived, dtype: int64

In [23]:
X_train.columns

Index(['Pclass', 'Sex_male', 'Age', 'Fare', 'Embarked_Q', 'Embarked_S'], dtype='object')

## gbt cv

In [23]:
# eta-- learning rate

#gamma-- gamma :(min_split_loss) Minimum loss reduction required to make a further partition on a leaf node of the tree. Higher
# gamma will resulot in more aggresive pruning and prevent Overfitting or may led to an underfit model

#n_estimators- Number of boosting rounds.

#lambda-- 

# New Prediction= Previous Prediction + alpha* output

In [27]:
gbt=GradientBoostingClassifier()
params={'max_depth':[10,20,30,50],
       'n_estimators':[100,200],
       'learning_rate':[0.01,0.1]}

In [28]:
%%time
mod_cv=GridSearchCV(estimator=gbt,
            param_grid=params,
            scoring='accuracy',
            cv=3,
            verbose=1)
mod_cv.fit(X_train,y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Wall time: 24.1 s


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1],
                         'max_depth': [10, 20, 30, 50],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=1)

In [29]:
mod_cv.best_params_,mod_cv.best_score_

({'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 200},
 0.8019832405536054)

In [30]:
pd.DataFrame(mod_cv.cv_results_).iloc[:,6:]

,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,100,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.785714,0.801688,0.789030,0.792144,0.006883,3
1,200,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.789916,0.814346,0.801688,0.801983,0.009976,1
2,100,"{'learning_rate': 0.01, 'max_depth': 20, 'n_es...",0.743697,0.767932,0.776371,0.762667,0.013849,14
3,200,"{'learning_rate': 0.01, 'max_depth': 20, 'n_es...",0.743697,0.763713,0.776371,0.761261,0.013451,16
4,100,"{'learning_rate': 0.01, 'max_depth': 30, 'n_es...",0.739496,0.767932,0.776371,0.761267,0.015775,15
5,200,"{'learning_rate': 0.01, 'max_depth': 30, 'n_es...",0.743697,0.763713,0.780591,0.762667,0.015080,13
6,100,"{'learning_rate': 0.01, 'max_depth': 50, 'n_es...",0.743697,0.767932,0.780591,0.764074,0.015307,8
7,200,"{'learning_rate': 0.01, 'max_depth': 50, 'n_es...",0.743697,0.767932,0.780591,0.764074,0.015307,8
8,100,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.777311,0.805907,0.793249,0.792156,0.011700,2
9,200,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.768908,0.805907,0.793249,0.789355,0.015354,4


In [31]:
best_gbt=GradientBoostingClassifier(learning_rate=0.01,n_estimators=100,max_depth=10)
best_gbt.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.01, max_depth=10)

In [32]:
pred=best_gbt.predict(X_test)

In [33]:
accuracy_score(y_test,pred)

0.7653631284916201